In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:

import os

os.chdir("..")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/pytorch-template-aws


In [3]:
import os
import shutil
from pathlib import Path
import torch
import lightning as L
from lightning.pytorch.loggers import Logger
from typing import List
from src.datamodules.catdog_datamodule import CatDogImageDataModule
from src.utils.logging_utils import setup_logger, task_wrapper
from loguru import logger
from dotenv import load_dotenv, find_dotenv
import rootutils
import hydra
from omegaconf import DictConfig, OmegaConf
from lightning.pytorch.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    RichModelSummary,
    RichProgressBar,
)
from lightning.pytorch.loggers import TensorBoardLogger, CSVLogger

/anaconda/envs/emlo_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load environment variables
load_dotenv(find_dotenv(".env"))

# Setup root directory
try:
    root = rootutils.setup_root(__file__, indicator=".project-root")

except Exception as e:
    logger.error(e)
    root = Path(os.getcwd())
    os.environ["PROJECT_ROOT"] = str(root)

2024-11-08 18:25:17.572 | ERROR    | __main__:<module>:9 - name '__file__' is not defined


In [5]:
def load_checkpoint_if_available(ckpt_path: str) -> str:
    """Check if the specified checkpoint exists and return the valid checkpoint path."""
    if ckpt_path and Path(ckpt_path).exists():
        logger.info(f"Checkpoint found: {ckpt_path}")
        return ckpt_path
    else:
        logger.warning(
            f"No checkpoint found at {ckpt_path}. Using current model weights."
        )
        return None


def clear_checkpoint_directory(ckpt_dir: str):
    """Clear all contents of the checkpoint directory without deleting the directory itself."""
    ckpt_dir_path = Path(ckpt_dir)
    if ckpt_dir_path.exists() and ckpt_dir_path.is_dir():
        logger.info(f"Clearing checkpoint directory: {ckpt_dir}")
        # Iterate over all files and directories in the checkpoint directory and remove them
        for item in ckpt_dir_path.iterdir():
            try:
                if item.is_file() or item.is_symlink():
                    item.unlink()  # Remove file or symlink
                elif item.is_dir():
                    shutil.rmtree(item)  # Remove directory
            except Exception as e:
                logger.error(f"Failed to delete {item}: {e}")
        logger.info(f"Checkpoint directory cleared: {ckpt_dir}")
    else:
        logger.info(
            f"Checkpoint directory does not exist. Creating directory: {ckpt_dir}"
        )
        os.makedirs(ckpt_dir_path, exist_ok=True)


@task_wrapper
def train_module(
    cfg: DictConfig,
    data_module: L.LightningDataModule,
    model: L.LightningModule,
    trainer: L.Trainer,
):
    """Train the model using the provided Trainer and DataModule."""
    logger.info("Training the model")
    trainer.fit(model, data_module)
    train_metrics = trainer.callback_metrics
    try:
        logger.info(
            f"Training completed with the following metrics- train_acc: {train_metrics['train_acc'].item()} and val_acc: {train_metrics['val_acc'].item()}"
        )
    except KeyError:
        logger.info(f"Training completed with the following metrics:{train_metrics}")

    return train_metrics


@task_wrapper
def run_test_module(
    cfg: DictConfig,
    datamodule: L.LightningDataModule,
    model: L.LightningModule,
    trainer: L.Trainer,
):
    """Test the model using the best checkpoint or the current model weights."""
    logger.info("Testing the model")
    datamodule.setup(stage="test")

    ckpt_path = load_checkpoint_if_available(cfg.ckpt_path)

    # If no checkpoint is available, Lightning will use current model weights
    test_metrics = trainer.test(model, datamodule, ckpt_path=ckpt_path)
    logger.info(f"Test metrics:\n{test_metrics}")

    return test_metrics[0] if test_metrics else {}

In [6]:
import hydra
from omegaconf import DictConfig, OmegaConf


# Function to load the configuration as an object without using the @hydra.main decorator
def load_config() -> DictConfig:
    # Initialize the configuration context (e.g., "../configs" directory)
    with hydra.initialize(config_path="../configs"):
        # Compose the configuration object with a specific config name (e.g., "train")
        cfg = hydra.compose(config_name="train")
    return cfg


# Load the configuration
cfg = load_config()

# Print the entire configuration for reference
print("Full Configuration:")
print(OmegaConf.to_yaml(cfg))

/tmp/ipykernel_487789/541470590.py:8: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../configs"):


Full Configuration:
task_name: train
tags:
- dev
train: true
test: false
ckpt_path: ${paths.ckpt_dir}/best-checkpoint.ckpt
seed: 42
name: catdog_experiment
data:
  _target_: src.datamodules.catdog_datamodule.CatDogImageDataModule
  data_dir: ${paths.data_dir}
  url: ${paths.data_url}
  num_workers: 8
  batch_size: 64
  train_val_split:
  - 0.8
  - 0.2
  pin_memory: true
  image_size: 160
model:
  _target_: src.models.catdog_model.ViTTinyClassifier
  img_size: 160
  patch_size: 16
  num_classes: 2
  embed_dim: 64
  depth: 6
  num_heads: 2
  mlp_ratio: 3
  pre_norm: false
  lr: 0.001
  weight_decay: 1.0e-05
  factor: 0.1
  patience: 10
  min_lr: 1.0e-06
callbacks:
  model_checkpoint:
    dirpath: ${paths.ckpt_dir}
    filename: best-checkpoint
    monitor: val_acc
    verbose: true
    save_last: true
    save_top_k: 1
    mode: max
    auto_insert_metric_name: false
    save_weights_only: false
    every_n_train_steps: null
    train_time_interval: null
    every_n_epochs: null
    save

In [7]:
# Initialize logger
if cfg.task_name == "train":
    log_path = Path(cfg.paths.log_dir) / "train.log"
else:
    log_path = Path(cfg.paths.log_dir) / "eval.log"
setup_logger(log_path)

logger.info(f"Whole Config:\n{OmegaConf.to_yaml(cfg)}")

2024-11-08 18:25:23 | INFO     | __main__:<module>:8 - Whole Config:
task_name: train
tags:
- dev
train: true
test: false
ckpt_path: ${paths.ckpt_dir}/best-checkpoint.ckpt
seed: 42
name: catdog_experiment
data:
  _target_: src.datamodules.catdog_datamodule.CatDogImageDataModule
  data_dir: ${paths.data_dir}
  url: ${paths.data_url}
  num_workers: 8
  batch_size: 64
  train_val_split:
  - 0.8
  - 0.2
  pin_memory: true
  image_size: 160
model:
  _target_: src.models.catdog_model.ViTTinyClassifier
  img_size: 160
  patch_size: 16
  num_classes: 2
  embed_dim: 64
  depth: 6
  num_heads: 2
  mlp_ratio: 3
  pre_norm: false
  lr: 0.001
  weight_decay: 1.0e-05
  factor: 0.1
  patience: 10
  min_lr: 1.0e-06
callbacks:
  model_checkpoint:
    dirpath: ${paths.ckpt_dir}
    filename: best-checkpoint
    monitor: val_acc
    verbose: true
    save_last: true
    save_top_k: 1
    mode: max
    auto_insert_metric_name: false
    save_weights_only: false
    every_n_train_steps: null
    train_time

In [8]:
# the path to the checkpoint directory
root_dir = cfg.paths.root_dir
logger.info(f"Root directory: {root_dir}")

logger.info(f"Current working directory: {os.listdir(root_dir)}")

ckpt_dir = cfg.paths.ckpt_dir
logger.info(f"Checkpoint directory: {ckpt_dir}")

# the path to the data directory
data_dir = cfg.paths.data_dir
logger.info(f"Data directory: {data_dir}")

# the path to the log directory
log_dir = cfg.paths.log_dir
logger.info(f"Log directory: {log_dir}")

# the path to the artifact directory
artifact_dir = cfg.paths.artifact_dir
logger.info(f"Artifact directory: {artifact_dir}")

# output directory
# output_dir = cfg.paths.output_dir
# logger.info(f"Output directory: {output_dir}")

# name of the experiment
experiment_name = cfg.name
logger.info(f"Experiment name: {experiment_name}")


2024-11-08 18:25:25 | INFO     | __main__:<module>:3 - Root directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/pytorch-template-aws
2024-11-08 18:25:25 | INFO     | __main__:<module>:5 - Current working directory: ['.dvc', '.dvcignore', '.env', '.git', '.github', '.gitignore', '.project-root', 'aws', 'basic_setup.md', 'configs', 'data', 'data.dvc', 'docker-compose.yaml', 'Dockerfile', 'ec2_runner_setup.md', 'logs', 'main.py', 'notebooks', 'poetry.lock', 'pyproject.toml', 'README.md', 'setup_aws_ci.md', 'src', 'tests', 'todo.md']
2024-11-08 18:25:25 | INFO     | __main__:<module>:8 - Checkpoint directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/pytorch-template-aws/checkpoints
2024-11-08 18:25:25 | INFO     | __main__:<module>:12 - Data directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/soutrik-vm-dev/code/Users/Soutrik.Chowdhury/pytorch-template-aws/data/
2024-11-08 18:25:25 

In [9]:
# Initialize DataModule
logger.info(f"Instantiating datamodule <{cfg.data._target_}>")
datamodule: L.LightningDataModule = hydra.utils.instantiate(cfg.data)

2024-11-08 18:25:28 | INFO     | __main__:<module>:2 - Instantiating datamodule <src.datamodules.catdog_datamodule.CatDogImageDataModule>


In [10]:
# Check for GPU availability
logger.info("GPU available" if torch.cuda.is_available() else "No GPU available")

# Set seed for reproducibility
L.seed_everything(cfg.seed, workers=True)

2024-11-08 18:25:28 | INFO     | __main__:<module>:2 - No GPU available
Seed set to 42


42

In [11]:
# Initialize model
logger.info(f"Instantiating model <{cfg.model._target_}>")
model: L.LightningModule = hydra.utils.instantiate(cfg.model)

2024-11-08 18:25:29 | INFO     | __main__:<module>:2 - Instantiating model <src.models.catdog_model.ViTTinyClassifier>


In [12]:
logger.info(f"Model summary:\n{model}")

2024-11-08 18:25:30 | INFO     | __main__:<module>:1 - Model summary:
ViTTinyClassifier(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 64, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=64, out_features=192, bias=False)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=64, out_features=64, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=64, out

In [13]:
def initialize_callbacks(cfg: DictConfig) -> List[L.Callback]:
    """Initialize the callbacks based on the configuration."""
    if not cfg:
        logger.warning("No callback configs found! Skipping..")
        return callbacks

    if not isinstance(cfg, DictConfig):
        raise TypeError("Callbacks config must be a DictConfig!")
    callbacks = []

    # Initialize the model checkpoint callback
    model_checkpoint = ModelCheckpoint(**cfg.callbacks.model_checkpoint)
    callbacks.append(model_checkpoint)

    # Initialize the early stopping callback
    early_stopping = EarlyStopping(**cfg.callbacks.early_stopping)
    callbacks.append(early_stopping)

    # Initialize the rich model summary callback
    model_summary = RichModelSummary(**cfg.callbacks.rich_model_summary)
    callbacks.append(model_summary)

    # Initialize the rich progress bar callback
    progress_bar = RichProgressBar(**cfg.callbacks.rich_progress_bar)
    callbacks.append(progress_bar)

    return callbacks


def initialize_logger(cfg: DictConfig) -> Logger:
    """Initialize the logger based on the configuration."""
    if not cfg:
        logger.warning("No logger configs found! Skipping..")
        return None

    if not isinstance(cfg, DictConfig):
        raise TypeError("Logger config must be a DictConfig!")

    loggers = []

    # Initialize the TensorBoard logger
    tensorboard_logger = TensorBoardLogger(**cfg.loggers.tensorboard)
    loggers.append(tensorboard_logger)

    # Initialize the CSV logger
    csv_logger = CSVLogger(**cfg.loggers.csv)
    loggers.append(csv_logger)

    return loggers

In [1]:
import timm
print(timm.list_models())

/anaconda/envs/emlo_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['bat_resnext26ts', 'beit_base_patch16_224', 'beit_base_patch16_384', 'beit_large_patch16_224', 'beit_large_patch16_384', 'beit_large_patch16_512', 'beitv2_base_patch16_224', 'beitv2_large_patch16_224', 'botnet26t_256', 'botnet50ts_256', 'caformer_b36', 'caformer_m36', 'caformer_s18', 'caformer_s36', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384', 'cait_s36_384', 'cait_xs24_384', 'cait_xxs24_224', 'cait_xxs24_384', 'cait_xxs36_224', 'cait_xxs36_384', 'coat_lite_medium', 'coat_lite_medium_384', 'coat_lite_mini', 'coat_lite_small', 'coat_lite_tiny', 'coat_mini', 'coat_small', 'coat_tiny', 'coatnet_0_224', 'coatnet_0_rw_224', 'coatnet_1_224', 'coatnet_1_rw_224', 'coatnet_2_224', 'coatnet_2_rw_224', 'coatnet_3_224', 'coatnet_3_rw_224', 'coatnet_4_224', 'coatnet_5_224', 'coatnet_bn_0_rw_224', 'coatnet_nano_cc_224', 'coatnet_nano_rw_224', 'coatnet_pico_rw_224', 'coatnet_rmlp_0_rw_224', 'coatnet_rmlp_1_rw2_224', 'coatnet_rmlp_1_rw_224', 'coatnet_rmlp_2_rw_224', 'coatnet_rmlp_2

##### testing the litserve model

In [2]:
import requests
from urllib.request import urlopen
import base64

In [33]:
url = "https://media.istockphoto.com/id/541844008/photo/portland-grand-floral-parade-2016.jpg?s=2048x2048&w=is&k=20&c=ZuvR6oDv5WxwL5dhXKAbevysEXhXV47shJdpzkqen5Y="
img_data = urlopen(url).read()
print(type(img_data))

<class 'bytes'>


In [34]:
# Convert to base64 string
img_bytes = base64.b64encode(img_data).decode('utf-8')
print(type(img_bytes))

<class 'str'>


In [35]:
response = requests.post(
    "http://localhost:8080/predict", json={"image": img_bytes}  # image is the key
)

In [36]:
if response.status_code == 200:
    predictions = response.json()["predictions"]
    print("\\nTop 5 Predictions:")
    for pred in predictions:
        print(f"{pred['label']}: {pred['probability']:.2%}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

\nTop 5 Predictions:
mountain_bike, all-terrain_bike, off-roader: 82.13%
maillot: 5.09%
crash_helmet: 1.84%
bicycle-built-for-two, tandem_bicycle, tandem: 1.83%
alp: 0.69%


########################################## End of the script ##########################################